In [106]:
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
from linearmodels.panel import PanelOLS


In [107]:
df = pd.read_csv('data/clean/filtered_data.csv')

In [108]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112698 entries, 0 to 112697
Data columns (total 48 columns):
 #   Column                                                                          Non-Null Count   Dtype  
---  ------                                                                          --------------   -----  
 0   Date                                                                            112698 non-null  object 
 1   Price                                                                           112698 non-null  float64
 2   Twitter Followers 24h                                                           112698 non-null  float64
 3   Gini Index                                                                      112698 non-null  float64
 4   Age Consumed                                                                    112698 non-null  float64
 5   Dormant Circulation (90d)                                                       112698 non-null  float64
 6   Acti

In [109]:
# Convert Date column to datetime if not already
df['Date'] = pd.to_datetime(df['Date'])

# Extract the week (Period) for grouping
df['Week'] = df['Date'].dt.to_period('W')

# Aggregate the data by ticker and week (excluding Week column to avoid conflict)
aggregated_df = (
    df.groupby(['ticker', 'Week'])
    .agg({col: 'mean' for col in df.columns if col not in ['Price', 'Date', 'ticker', 'Week']})
    .reset_index()
)

# Add the last value of Price for each week
price_last = df.groupby(['ticker', 'Week'])['Price'].last().reset_index()

# Merge the aggregated data with the weekly Price
aggregated_df = aggregated_df.merge(price_last, on=['ticker', 'Week'])

# Ensure data is sorted by ticker and week
aggregated_df = aggregated_df.sort_values(by=['ticker', 'Week']).reset_index(drop=True)


/var/folders/7m/xypr2v416g37k4tj5sf58x4h0000gn/T/ipykernel_45096/760841287.py:5: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['Week'] = df['Date'].dt.to_period('W')


In [110]:
df = aggregated_df
df['Date'] = df['Week'].dt.to_timestamp()
df.drop(columns=['Week'], inplace=True)

In [111]:
# Apply signed log transformation
cols_to_transform = df.columns.drop(['Date', 'ticker'])
for col in cols_to_transform:
    df[col] = np.sign(df[col]) * np.log1p(abs(df[col]))

# Prepare the data
df['Date'] = pd.to_datetime(df['Date']).dt.tz_localize(None)
df = df.sort_values(by=['ticker', 'Date']).reset_index(drop=True)
df['token_age'] = df.groupby('ticker')['Date'].transform(lambda x: (x - x.min()).dt.days // 7)
df['token_age'] = df['token_age'].astype(float)

# Log-transform volume if available
if 'Transaction Volume USD' in df.columns:
    df['Transaction Volume USD'] = np.sign(df['Transaction Volume USD']) * np.log1p(abs(df['Transaction Volume USD']))

# Create weekly time dummies
df['Week'] = df['Date'].dt.to_period('W')
# time_dummies = pd.get_dummies(df['Week'], prefix='Week')
# df = pd.concat([df, time_dummies], axis=1)






In [112]:
df.Date

0       2024-09-23
1       2024-09-30
2       2024-10-07
3       2024-10-14
4       2024-10-21
           ...    
18271   2024-10-21
18272   2024-10-28
18273   2024-11-04
18274   2024-11-11
18275   2024-11-18
Name: Date, Length: 18276, dtype: datetime64[ns]

In [113]:
# Filter independent variables by correlation threshold
independent_vars = df.drop(columns=['Price', 'Date', 'ticker'])
correlation_threshold = 0.95
correlation_matrix = independent_vars.corr()
upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
columns_to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column].abs() > correlation_threshold)]
independent_vars_filtered = independent_vars.drop(columns=columns_to_drop)

# Lag independent variables
for col in independent_vars_filtered.columns:
    df[col] = df.groupby('ticker')[col].shift(1)

# df = df.dropna(subset=independent_vars_filtered.columns)
# df.head()

/var/folders/7m/xypr2v416g37k4tj5sf58x4h0000gn/T/ipykernel_45096/833744407.py:4: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix = independent_vars.corr()


In [114]:
# # Clean independent_vars_filtered for VIF calculation
# independent_vars_filtered.replace([np.inf, -np.inf], np.nan, inplace=True)
# independent_vars_filtered.dropna(inplace=True)

# Check for non-numeric columns and drop if necessary
independent_vars_filtered = independent_vars_filtered.select_dtypes(include=[np.number])

# Calculate VIF
vif_data = pd.DataFrame()
vif_data['Variable'] = independent_vars_filtered.columns
vif_data['VIF'] = [variance_inflation_factor(independent_vars_filtered.values, i) for i in range(independent_vars_filtered.shape[1])]

# Drop variables with high VIF
high_vif_vars = vif_data[vif_data['VIF'] > 10]['Variable']
independent_vars_filtered = independent_vars_filtered.drop(columns=high_vif_vars)


In [115]:
# Create month-year combination column
df['MonthYear'] = df['Date'].dt.to_period('M')

# Generate dummies for MonthYear
month_year_dummies = pd.get_dummies(df['MonthYear'], prefix='MonthYear')

# Concatenate dummies to the main DataFrame
df = pd.concat([df, month_year_dummies], axis=1)

# Ensure MultiIndex is set for the main DataFrame
df = df.set_index(['ticker', 'Date'])

# Ensure the filtered independent variables have the same index as df
independent_vars_filtered.index = df.index
# Assign the same MultiIndex to month_year_dummies
month_year_dummies.index = df.index
# Drop the first MonthYear dummy to avoid multicollinearity
month_year_dummies = month_year_dummies.iloc[:, 1:]
# Combine dependent and independent variables, including dummies
panel_data = pd.concat([df[['Price', 'Transaction Volume USD']], independent_vars_filtered, month_year_dummies], axis=1)

# Drop any rows with NaNs
panel_data = panel_data.dropna()




In [116]:
panel_data

Price  Transaction Volume USD  Twitter Followers 24h  \
ticker Date                                                                  
AAVE   2024-09-30  4.945995                3.031496               4.550865   
       2024-10-07  5.055868                3.027745               5.408452   
       2024-10-14  5.013871                3.029628               4.856485   
       2024-10-21  5.057919                3.024800              -1.860752   
       2024-10-28  5.162461                3.023846              -4.353039   
...                     ...                     ...                    ...   
ZRX    2024-10-21  0.291928                2.904824              -3.610918   
       2024-10-28  0.281364                2.908917              -3.953987   
       2024-11-04  0.306959                2.867996               2.344549   
       2024-11-11  0.358314                2.893774              -2.472930   
       2024-11-18  0.358314                2.948336               4.934474   

                   Gini Index  \
ticker Date                     
AAVE   2024-09-30    0.686496   
       2024-10-07    0.686478   
       2024-10-14    0.686479   
       2024-10-21    0.686477   
       2024-10-28    0.686452   
...                       ...   
ZRX    2024-10-21    0.688004   
       2024-10-28    0.688015   
       2024-11-04    0.688030   
       2024-11-11    0.688129   
       2024-11-18    0.688197   

                   The Ratio of Daily On-Chain Transaction Volume in Profit to Loss  \
ticker Date                                                                           
AAVE   2024-09-30                                           0.680636                  
       2024-10-07                                           0.908721                  
       2024-10-14                                           1.371523                  
       2024-10-21                                           1.195723                  
       2024-10-28                                           0.746754                  
...                                                              ...                  
ZRX    2024-10-21                                           1.608492                  
       2024-10-28                                           0.551764                  
       2024-11-04                                           0.967326                  
       2024-11-11                                           1.032922                  
       2024-11-18                                           0.891546                  

                   Mean Coin Age  Mean Dollar Invested Age  \
ticker Date                                                  
AAVE   2024-09-30       6.530459                  6.447942   
       2024-10-07       6.537682                  6.452578   
       2024-10-14       6.499465                  6.416756   
       2024-10-21       6.471867                  6.391448   
       2024-10-28       6.477466                  6.394440   
...                          ...                       ...   
ZRX    2024-10-21       6.911180                  6.822876   
       2024-10-28       6.917296                  6.830739   
       2024-11-04       6.923177                  6.836766   
       2024-11-11       6.924689                  6.836692   
       2024-11-18       6.924033                  6.835094   

                   MVRV Long/Short Difference  MVRV Ratio  MVRV Ratio (1d)  \
ticker Date                                                                  
AAVE   2024-09-30                    0.241316    0.333710        -0.018197   
       2024-10-07                    0.280622    0.396367        -0.006572   
       2024-10-14                    0.288325    0.388295         0.005038   
       2024-10-21                    0.278994    0.370998         0.012379   
       2024-10-28                    0.296041    0.446785         0.039169   
...                                       ...         ...              ...   
ZRX    2024-10

In [118]:
# Define dependent and independent variables
dependent = panel_data['Price']
independent = panel_data.drop(columns=['Price'])

# Add a constant term to the independent variables
independent = independent.assign(constant=1)

# Run the fixed-effects regression
fixed_effects_model = PanelOLS(dependent, independent, entity_effects=True)
results = fixed_effects_model.fit()

# Display results
print(results.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                  Price   R-squared:                        0.4795
Estimator:                   PanelOLS   R-squared (Between):              0.0158
No. Observations:               18183   R-squared (Within):               0.4795
Date:                Sat, Nov 23 2024   R-squared (Overall):              0.0614
Time:                        19:08:44   Log-likelihood                   -9048.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      162.44
Entities:                          93   P-value                           0.0000
Avg Obs:                       195.52   Distribution:               F(102,17988)
Min Obs:                       8.0000                                           
Max Obs:                       365.00   F-statistic (robust):             162.44
                            

Consider adopting an IV approach if you suspect that on-chain metrics (e.g., MVRV) might be influenced by price changes.

Food for thought: The authors in the original paper are studying the effect of dev activity on the price. On-Chain metrics most likely dont have an effect on the price. They are indicators of underlying mechanisms that we are interested in studying. What are these? E.g. the effect of people being in profit, the effect of people holding duration, the effect of network activity? 
